In [ ]:
#import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#web scraping with XPath and Selenium
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time


#scrape from url 
url = #[url of apartment website]
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

#will send a text to [my phone number] using twilio if an apartment I want is available
from twilio.rest import Client
twilio_account_sid = #[my twilio account sid]
twilio_auth_token = #[my twilio auth token]
twilio_phone_sender = #[my twilio phone number]
twilio_phone_recipient = #[my personal phone number]

def send_text_alert(alert_str):
    client = Client(twilio_account_sid, twilio_auth_token)
    client.messages.create(to=twilio_phone_recipient, 
    from_=twilio_phone_sender, body=alert_str)



In [ ]:
apts = [102, 210, 211, 215, 217, 311, 313, 315, 317] #list of apartments I want to check

#The xpaths of the cells in the table which says whether or not each apartment is available are extremely similar; they just differ by a single number in the middle of the xpath.
xpath_nums = {102: 3, 210: 15, 211: 16, 215: 18, 217: 19, 311: 28, 313: 29, 315: 30, 317: 31} #list of xpath numbers for each apartment
xpaths = {}
for num in xpath_nums.keys():
    xpaths[num] = f'//*[@id="block-views-our-apartments-block-1"]/div/div/div[2]/div[2]/div/ul/li[{xpath_nums[num]}]/div/span/div[5]/a/span[1]'


#given an apartment number, check_apt will check if it is available and send a text if it is
def check_apt(apt):
    element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpaths[apt])))
    if element.text != 'RENTED':
        body = "Apt. " + str(apt) + " is available!"
        send_text_alert(body)
        return True
    else:
        return False

In [ ]:
import logging
import os
import time
DELAY_TIME = 600 #seconds

In [ ]:
#maintains a log of whether or not each apartment is available every 10 minutes
log = logging.getLogger(__name__)
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"), format='%(asctime)s %(message)s')
log.info("Apt Monitor")
while True:
    url = #[url of apartment website]
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    try: 
        for apt in apts:
            if check_apt(apt):
                log.info("Apt. " + str(apt) + " is available!")
            else:
                log.info("Apt. " + str(apt) + " is not available.")
    except:
        log.info("Error checking website")
    driver.quit()
    time.sleep(DELAY_TIME)